<center>
<hr>
<h2>Predict The Bay Area’s Home Prices</h2>
<hr>
</center>

*  you will investigate the housing price dataset from Bay Area Home Sales Database and Zillow to Predict The Bay Area’s Home Prices.
* This dataset was based on the homes sold between January 2013 and December 2015.
* Please use all the techniques we have learned in the class to preprocesss/clean the dataset.
* You can drop redundant features.
<p style="color:blue"><b>final_data.csv</b></p>
* Please apply the folowing two methods to Predict The Bay Area’s Home Prices.

<h3>Method 1: Multiple Linear Regression</h3>
* Bulding the optimal model using Automatic implementations of Backward Elimination
```
def backwardElimination(x, sl):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(y, x).fit()
        maxVar = max(regressor_OLS.pvalues)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j] == maxVar):
                    x = np.delete(x, j, 1)
    regressor_OLS.summary()
    return x
```
* Split the dataset into training sets and test sets
* Fit LinearRegression to the training sets 
* Print the Linear Regression R squared score on the test sets
* Print the Linear Regression RMSE score between on the test sets

<h3>Method 2: Least Absolute Shrinkage and Selection Operator(Lasso)</h3>
* Split the dataset into training sets and test sets
* Fit Lasso to the training sets. 
* Print the Lasso R squared score on the test sets
* Print the Lasso RMSE score between on the test sets


## 1. Import dataset "final_data.csv" in pandas

In [ ]:
import pandas as pd
dataset = pd.read_csv("final_data.csv")
print(dataset.head())

   Unnamed: 0                             address  \
0           2  Address: 1160 Mission Street #2007   
1           5       Address: 260 King Street #475   
2           7     Address: 560 Missouri Street #B   
3           9        Address: 350 Missouri Street   
4          11         Address: 3658 Folsom Street   

                                                info  \
0   San FranciscoSales price: 1300000Sales date: ...   
1   San FranciscoSales price: 750000Sales date: 0...   
2   San FranciscoSales price: 1495000Sales date: ...   
3   San FranciscoSales price: 2700000Sales date: ...   
4   San FranciscoSales price: 1530000Sales date: ...   

                   z_address  bathrooms  bedrooms  finishedsqft lastsolddate  \
0  1160 Mission St UNIT 2007        2.0       2.0        1043.0   02/17/2016   
1       260 King St UNIT 475        1.0       1.0         903.0   02/17/2016   
2        560 Missouri St # B        4.0       3.0        1425.0   02/17/2016   
3            350 Missour

## 2. Preprocess/Clean the Dataset

In [ ]:
count=0
for i in range(dataset.shape[1]):
    count=count+sum(dataset.iloc[:,i].isnull())
    #print ('Number of null values in',dataset.columns.values[i],'column is',count)
print("Number of Null values in the dataset is" , count)
print("shape of dataset is",dataset.shape)

Number of Null values in the dataset is 0
shape of dataset is (11330, 19)


In [ ]:
#dataset.dtypes
dataset["lastsolddate"]=pd.to_datetime(dataset["lastsolddate"])

dataset['lastSoldYearMonth'] = dataset['lastsolddate'].map(lambda x: 100*x.year + x.month)
#dataset['last_sold_year'] = dataset.lastsolddate.dt.year


dataset['Lot size']=dataset['info'].map(lambda x: x.split(":")[-1])       #getting lot size from info
dataset['Lot size']=pd.to_numeric(dataset['Lot size'])
#dataset['Lot size']=dataset['Lot size'].astype(str).astype(int)         #converting to type int
print(dataset['Lot size'].nunique())  


#dataset['info'] = dataset['info'].astype(str)
#dataset.info.apply(str)
#print(list(dataset['info'][0].split(":"))[-1])
#print(len(dataset))

#for i in range(len(dataset)):
#    dataset.loc[i,'Lot size']=list(dataset['info'][i].split(":"))[-1]
#dataset['Lot size']=pd.Series(list(dataset.loc[:,'info'].str.split(":"))[-1])
#dataset['Lot size']=list(dataset.loc[:,'info'].str.split(":"))[-1]
#print(dataset.head(3))
dataset.dtypes

2558


Unnamed: 0                    int64
address                      object
info                         object
z_address                    object
bathrooms                   float64
bedrooms                    float64
finishedsqft                float64
lastsolddate         datetime64[ns]
lastsoldprice               float64
latitude                    float64
longitude                   float64
neighborhood                 object
totalrooms                  float64
usecode                      object
yearbuilt                   float64
zestimate                   float64
zindexvalue                  object
zipcode                     float64
zpid                        float64
lastSoldYearMonth             int64
Lot size                      int64
dtype: object

In [ ]:
print(dataset.neighborhood.nunique())              #no of diff values in column neighborhood is 71

from sklearn.preprocessing import LabelEncoder              #So lets use this column values by encoding
lblEncoder = LabelEncoder()
dataset['neighborhood']=lblEncoder.fit_transform(dataset['neighborhood'])


71


In [ ]:
print(dataset.usecode.nunique())                  #no of diff values in column usecode is 10

from sklearn.preprocessing import LabelEncoder              #So lets use this column values by encoding
lblEncoder = LabelEncoder()
dataset['usecode']=lblEncoder.fit_transform(dataset['usecode'])


10


In [ ]:
#dataset['zindexvalue']=dataset['zindexvalue'].map(lambda x: x)      
#removing ',' in zindecvalue
dataset["zindexvalue"] = dataset["zindexvalue"].str.replace(',','')
dataset["zindexvalue"] = pd.to_numeric(dataset["zindexvalue"])


print(dataset.head(3))

dataset.dtypes

   Unnamed: 0                             address  \
0           2  Address: 1160 Mission Street #2007   
1           5       Address: 260 King Street #475   
2           7     Address: 560 Missouri Street #B   

                                                info  \
0   San FranciscoSales price: 1300000Sales date: ...   
1   San FranciscoSales price: 750000Sales date: 0...   
2   San FranciscoSales price: 1495000Sales date: ...   

                   z_address  bathrooms  bedrooms  finishedsqft lastsolddate  \
0  1160 Mission St UNIT 2007        2.0       2.0        1043.0   2016-02-17   
1       260 King St UNIT 475        1.0       1.0         903.0   2016-02-17   
2        560 Missouri St # B        4.0       3.0        1425.0   2016-02-17   

   lastsoldprice   latitude    ...     neighborhood  totalrooms  usecode  \
0      1300000.0  37.778705    ...               59         4.0        1   
1       750000.0  37.777641    ...               59         3.0        1   
2      149500

Unnamed: 0                    int64
address                      object
info                         object
z_address                    object
bathrooms                   float64
bedrooms                    float64
finishedsqft                float64
lastsolddate         datetime64[ns]
lastsoldprice               float64
latitude                    float64
longitude                   float64
neighborhood                  int64
totalrooms                  float64
usecode                       int64
yearbuilt                   float64
zestimate                   float64
zindexvalue                   int64
zipcode                     float64
zpid                        float64
lastSoldYearMonth             int64
Lot size                      int64
dtype: object

In [ ]:
X=dataset.drop(['address', 'info', 'z_address','Unnamed: 0' ,'lastsolddate' ], axis=1)

print("hi")

print(X.head(3))

hi
   bathrooms  bedrooms  finishedsqft  lastsoldprice   latitude   longitude  \
0        2.0       2.0        1043.0      1300000.0  37.778705 -122.412635   
1        1.0       1.0         903.0       750000.0  37.777641 -122.393417   
2        4.0       3.0        1425.0      1495000.0  37.759198 -122.396516   

   neighborhood  totalrooms  usecode  yearbuilt  zestimate  zindexvalue  \
0            59         4.0        1     2007.0  1167508.0       975700   
1            59         3.0        1     2004.0   823719.0       975700   
2            53         6.0        1     2003.0  1708594.0      1277600   

   zipcode        zpid  lastSoldYearMonth  Lot size  
0  94103.0  83152781.0             201602     43557  
1  94107.0  69819817.0             201602    213232  
2  94107.0  64972847.0             201602      2496  


In [ ]:
X = X.drop('lastsoldprice', axis=1)
y = dataset['lastsoldprice']

print(X.head(3))

X.dtypes

   bathrooms  bedrooms  finishedsqft   latitude   longitude  neighborhood  \
0        2.0       2.0        1043.0  37.778705 -122.412635            59   
1        1.0       1.0         903.0  37.777641 -122.393417            59   
2        4.0       3.0        1425.0  37.759198 -122.396516            53   

   totalrooms  usecode  yearbuilt  zestimate  zindexvalue  zipcode  \
0         4.0        1     2007.0  1167508.0       975700  94103.0   
1         3.0        1     2004.0   823719.0       975700  94107.0   
2         6.0        1     2003.0  1708594.0      1277600  94107.0   

         zpid  lastSoldYearMonth  Lot size  
0  83152781.0             201602     43557  
1  69819817.0             201602    213232  
2  64972847.0             201602      2496  


bathrooms            float64
bedrooms             float64
finishedsqft         float64
latitude             float64
longitude            float64
neighborhood           int64
totalrooms           float64
usecode                int64
yearbuilt            float64
zestimate            float64
zindexvalue            int64
zipcode              float64
zpid                 float64
lastSoldYearMonth      int64
Lot size               int64
dtype: object

## 3. Method 1: Multiple Linear Regression

In [ ]:
import statsmodels.api as sm
import numpy as np
def backwardElimination(x,y, sl):
  numVars = len(x[0])
  for i in range(0, numVars):
      regressor_OLS = sm.OLS(y, x.astype(float)).fit()
      maxVar = max(regressor_OLS.pvalues)
      if maxVar > sl:
          for j in range(0, numVars - i):
              if (regressor_OLS.pvalues[j] == maxVar):
                  x = np.delete(x, j, 1)
  print(regressor_OLS.summary())
  return x
train_X = backwardElimination(X.values,y,sl = 0.06)

                            OLS Regression Results                            
Dep. Variable:          lastsoldprice   R-squared:                       0.919
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                 1.429e+04
Date:                Sun, 03 Mar 2019   Prob (F-statistic):               0.00
Time:                        10:55:50   Log-Likelihood:            -1.6395e+05
No. Observations:               11330   AIC:                         3.279e+05
Df Residuals:                   11321   BIC:                         3.280e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1           6.47e+04   7054.271      9.172      0.0

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_X, y, test_size = 3)

# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
model_regressor = LinearRegression()
model_regressor.fit(X_train, y_train)

# Predicting the Test set results
#We can compare y_pred and y_test to find its accuracy
ypred = model_regressor.predict(X_test)

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

rmse = sqrt(mean_squared_error(y_test, ypred))
print("RMSE is:",rmse)

r2=r2_score(y_test, ypred)
print("r2 score:",r2)

RMSE is: 305241.53733734746
r2 score: 0.9186286865639133


## 4. Method 2: Least Absolute Shrinkage and Selection Operator(Lasso) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


from sklearn import linear_model
model_lasso = linear_model.Lasso(alpha=0.2)
#model_lasso = linear_model.Lasso()

model_lasso.fit(X_train,y_train)

ypred2=model_lasso.predict(X_test)

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

rmse2 = sqrt(mean_squared_error(y_test, ypred2))
print("RMSE is:",rmse2)

r22=r2_score(y_test, ypred2)
print("r2 score:",r22)

RMSE is: 461418.3343116447
r2 score: 0.7594900084273053


**5. Method 3: Decision Tree **

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state = 17)
model.fit(X_train,y_train)
target = model.predict(X_test)
DT_r2= r2_score(target,y_test)
DT_rmse = np.sqrt(mean_squared_error(target,y_test))
print("RMS: ",DT_rmse)
print("r2 score: ",DT_r2)

RMS:  505288.16355744563
r2 score:  0.7222667486444525


**6. Method 4: Random Forest **

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state = 17)
model.fit(X_train,y_train)
target = model.predict(X_test)
RF_r2= r2_score(target,y_test)
RF_rmse = np.sqrt(mean_squared_error(target,y_test))
print("RMS: ",RF_rmse)
print("r2 score: ",RF_r2)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RMS:  396494.77468316146
r2 score:  0.8016115214295947


## 7. Test Data Set Prediction Results Comparison Table of Method 1 and Method 2 
* Please fill in your results to the following table

<table>
<thead>
    <tr>
    <th colspan="3">Predict The Bay Area’s Home Prices</th>
    </tr>
    <tr>
    <th> </th>
    <th>Multiple Linear Regression</th>
    <th>Lasso</th>
    </tr>
</thead>
<tbody>
    <tr>
        <th>R Squared Score</th>
        <td> null </td>
        <td> null </td>
    </tr>
     <tr>
         <th>RMSE</th>
        <td> null </td>
        <td> null </td>
    </tr>
</tbody>
</table>

In [ ]:
from tabulate import tabulate
print(tabulate([['R squared Score', r2,r22,DT_r2,RF_r2], ['RMSE', rmse,rmse2,DT_rmse,RF_rmse]], headers=['Multiple Linear Regression', 'Lasso','Decision Tree','Random Forest']))

                   Multiple Linear Regression         Lasso    Decision Tree    Random Forest
---------------  ----------------------------  ------------  ---------------  ---------------
R squared Score                      0.918629       0.75949         0.722267         0.801612
RMSE                            305242         461418          505288           396495
